<a href="https://colab.research.google.com/github/tmendonca28/DLNID/blob/master/RNN_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pdb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from statistics import mean
import pandas as pd
import numpy as np
import torch.optim as optim

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
train_data = pd.read_csv("drive/My Drive/Masters/Project/DLNID/Datasets/KDDTrainRenameNominalValuesNormalized.csv", header=0, low_memory=False, dtype = np.float32)
test_data = pd.read_csv("drive/My Drive/Masters/Project/DLNID/Datasets/KDDTestRenameNominalValuesNormalized.csv", header=0, low_memory=False, dtype = np.float32)
print(len(train_data))
print(len(test_data))
print(test_data)

125973
22544
       duration  protocol_type  ...  dst_host_srv_rerror_rate  class
0      0.000000            1.0  ...                      1.00    1.0
1      0.000000            1.0  ...                      1.00    1.0
2      0.000035            1.0  ...                      0.00    0.0
3      0.000000            3.0  ...                      0.00    1.0
4      0.000017            1.0  ...                      0.71    1.0
5      0.000000            1.0  ...                      0.00    0.0
6      0.000000            1.0  ...                      0.04    0.0
7      0.000000            1.0  ...                      0.02    1.0
8      0.000000            1.0  ...                      0.00    0.0
9      0.000000            1.0  ...                      0.00    1.0
10     0.000000            1.0  ...                      0.32    1.0
11     0.000000            1.0  ...                      0.00    0.0
12     0.000000            1.0  ...                      1.00    1.0
13     0.000000      

In [0]:
targets_pd = train_data.iloc[0:, 41]
features_pd = train_data.iloc[0:, 0:41]

test_targets_pd = test_data.iloc[0:, 41]
test_features_pd = test_data.iloc[0:, 0:41]

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.FloatTensor(features_pd.values)
# torch_tensor = torch.tensor(targets_df['targets'].values)
targetsTrain = torch.LongTensor(targets_pd.values)

featuresTest = torch.FloatTensor(test_features_pd.values)
# torch_tensor = torch.tensor(targets_df['targets'].values)
targetsTest = torch.LongTensor(test_targets_pd.values)

# batch_size and epoch
batch_size = 1
# n_iters = 10000
num_epochs = 10

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest, targetsTest)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = False)
print(len(train))
print(len(test))
print(len(train_loader))
print(len(test_loader))

125973
22544
125973
22544


In [0]:
import argparse
parser = argparse.ArgumentParser(description='Network Intrusion Classifier')
parser.add_argument('--lr', action='store', type=float, help='use different versions of network', default=1e-1)
parser.add_argument('--epochs', action='store', type=int, help='use different versions of network', default=num_epochs)
opts = parser.parse_args('')
print(opts)

Namespace(epochs=10, lr=0.1)


In [0]:
# Defining the RNN Model
class RNNModel(nn.Module):
  def __init__(self):
    super(RNNModel, self).__init__()
    self.rnn = nn.LSTM(41, 80, 1)
    self.out = nn.Linear(80, 2)
    self.dropout = nn.Dropout(0.5)
    
    
  def forward(self, x):
    x = x.view(-1, x.shape[0], x.shape[1])
    out, (h_n, h_c) = self.rnn(x, None)
    out = self.out(out[-1, :, :])
    out = F.relu(out)
    out = self.dropout(out)
    out = torch.sigmoid(out)
    return out

In [0]:
def evaluate_network_performance(predictions, truth):
  # given a list with predicted and correct values
  # calculate precision, recall and f-score
#   pdb.set_trace()
  cm = get_confusion_matrix(predictions, truth)
  TP = cm[0][0]
  FP = cm[0][1]
  FN = cm[1][0]
  TN = cm[1][1]

  precision = TP/(TP+FP)
  recall = TP/(TP+FN)
  classification_accuracy = (TP+TN) / (TP + TN + FP + FN)
  f_score = 2 * ((precision*recall)/(precision + recall))

  return precision, recall, f_score, classification_accuracy

In [0]:
def get_confusion_matrix(preds, truth):
    K = len(np.unique(truth)) # Number of classes
    result = np.zeros((K, K))
    for i in range(len(truth)):
        result[preds[i]][truth[i]] += 1
    confusion_matrix = result
    return confusion_matrix

In [0]:
def test_network(model, test_set, targets):
  # given a dataset and network, compute accuracy
  
  criterion = nn.CrossEntropyLoss()
  
  labels = targets.tolist()
  
  test_loss = 0
  predictions = []
  
  with torch.no_grad():
    for i, (images, label) in enumerate(test_set):
      model.eval()
      
#       images, labels = images.to(device), labels.to(device)
      output = model(images)
      predicted_index = torch.max(output.data, 1)[1]
      predictions.append(predicted_index.item())
#       labels.append(label.item())
      
      test_loss = criterion(output, label).item()
    
    precision, recall, f_score, classification_accuracy = evaluate_network_performance(predictions, labels)

  return precision, recall, f_score, classification_accuracy

In [0]:
def train_network(model, train_dataset):
  optimizer = optim.Adam(model.parameters(), lr=opts.lr)
  criterion = nn.CrossEntropyLoss()
  
  num_training_epochs = opts.epochs
  
  for e in range(num_training_epochs):
    print("Running Epoch: ", e, " ....")
    running_loss = 0
    
    for i,(images, labels) in enumerate(train_dataset):
      model.train()
      optimizer.zero_grad()
#       images, labels = images.to(device), labels.to(device)
      
      output = model(images)
      loss = criterion(output, labels)
      
      running_loss += loss.item()
      
      loss.backward()
      optimizer.step()
      
    epoch_loss = running_loss/(len(train_dataset))
    
    print('Iteration ', e, 'loss ', epoch_loss)
    print('          PREC | REC | F1  |   ACC')
#     precision, recall, f_score, classification_accuracy = test_network(model, test_loader, targetsTest)
#     print('val set  ',"{0:.2f} | ".format(precision),"{0:.2f} | ".format(recall),"{0:.2f} | ".format(f_score),"{0:.2f}".format(classification_accuracy))

    precision, recall, f_score, classification_accuracy = test_network(model, train_dataset, targetsTrain)
    print('train set',"{0:.2f} | ".format(precision),"{0:.2f} | ".format(recall),"{0:.2f} | ".format(f_score),"{0:.2f}".format(classification_accuracy))
    print()

  return model

In [0]:
def main():
  model = RNNModel()
  print(model)
#   model.to(device)
  print('Started training network......')
  model = train_network(model, train_loader)
  
  print('Started testing network.......')
  precision, recall, f_score, classification_accuracy = test_network(model, test_loader, targetsTest)
  print('test set',"{0:.2f}".format(precision),"{0:.2f}".format(recall),"{0:.2f}".format(f_score),"{0:.2f}".format(classification_accuracy))
  print()

In [0]:
main()

RNNModel(
  (rnn): LSTM(41, 80)
  (out): Linear(in_features=80, out_features=2, bias=True)
  (dropout): Dropout(p=0.5)
)
Started training network......
Running Epoch:  0  ....


KeyboardInterrupt: ignored